In [1]:
import numpy as np
def y(ti, dti,la = None, lb = None, Na, Nb): 
    return Na*np.exp(-la*ti)*(1 - np.exp(-la*dti))+Nb*np.exp(-lb*ti)*(1 - np.exp(-lb*dti))

In [ ]:

la = 0.203
lb = 0.92
D = 0.001

la1 = la + D
lb2 = lb + D

V_l = (10**(-4))*np.array([[0.81, 0], [0, 3.24]])

N, V_y = curve_fit(y(la = la, lb = lb), (ti, dti), y_data, absolute_sigma = True, sigma = s_y)
N1, V_y1 = curve_fit(y(la = la1, lb = lb), (ti, dti), y_data, absolute_sigma = True, sigma = s_y)
N2, V_y2 = curve_fit(y(la = la, lb = lb2), (ti, dti), y_data, absolute_sigma = True, sigma = s_y)

N_a, N_b = N
N_1a, N_1b = N1
N_2a, N_2b = N 

partial_Na_la = (N_1a - N_a)/D
partial_Na_lb = (N_2a - N_a)/D

partial_Nb_la = (N_1b - N_b)/D
partial_Nb_lb = (N_2b - N_b)/D


